In [1]:
import os, sys, json
from preprocess.process_cypher import get_schema_from_json

### Test dataset builder --> done!

In [2]:

schema = get_schema_from_json(os.getcwd() + '/data/text2cypher/schema.json')

In [3]:
schema

{'flight_company': {'property_types': ['String',
   'String',
   'String',
   'String',
   'Long',
   'String',
   'String',
   'Long',
   'String',
   'String',
   'String',
   'Double',
   'Long',
   'String',
   'Long',
   'Long',
   'String',
   'Double',
   'String',
   'Double'],
  ':`flight_company.airport`': [{'name': ['Schiphol Airport',
     'Bergen Airport',
     'Akureyri Airport',
     'Baltimore-Washington Airport',
     'National Airport',
     'El Prat Airport',
     'Logan International Airport',
     'Billund Airport',
     'Ted Stevens Airport']},
   {'IATA': ['BOS', 'AEY', 'BLL', 'BWI', 'BGO', 'ANC', 'BRU', 'AMS', 'BCN']},
   {'Country': ['Denmark',
     'Netherlands',
     'Belgium',
     'United States',
     'Iceland',
     'Spain',
     'Norway']},
   {'ICAO': ['ENBR',
     'KBOS',
     'EBBR',
     'KBWI',
     'LEBL',
     'EHAM',
     'EKBI',
     'PANC',
     'BIAR']},
   {'id': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
   {'City': ['Brussels',
     'Bergen',
     'Barc

In [4]:
schema_cache = {}
for split in ['train.json', 'dev.json']:
    dev_fpath = os.getcwd() + '/data/text2cypher/'+ split
    with open(dev_fpath, 'r', encoding="utf-8") as f:
        data = json.load(f)
        print(len(data))
        for idx, sample in enumerate(data):
            db_id = sample["db_id"]
            if db_id == 'department_management':
                if db_id not in schema_cache:
                    schema_cache[db_id] = schema[db_id]
                every_schema = schema_cache[db_id]
                print(db_id)
                print(sample)
                print(every_schema)
                tags = list(every_schema.keys())[1:]
                for tag_id, tag in enumerate(tags):
                    property_name_value_pairs = every_schema[tag]
                    for pair in property_name_value_pairs:
                        for property_name, field_values in pair.items():
                            test = {
                                "db_id": db_id,
                                "query": sample["query"],
                                "question": sample["question"],
                                "answers": sample["answers"],
                                "db_tag_names": tags, 
                                "db_property_names": 
                                    [
                                        {
                                            "tag_id": tag_id, "property_name": property_name
                                        }
                                    ],
                                "db_property_types": every_schema['property_types']
                                }
                            print(test)

3175
department_management
{'question': 'How many heads of the departments are older than 56 ?', 'query': 'MATCH (head:`department_management.head`)\nWHERE head.age > 56\nRETURN count(*)', 'db_id': 'department_management', 'answers': [[5]]}
{'property_types': ['String', 'String', 'Long', 'Long', 'Double', 'Double', 'String', 'String', 'Double', 'Long', 'String'], ':`department_management.department`': [{'Name': ['Justice', 'Commerce', 'Interior', 'Education', 'Veterans Affairs', 'Energy', 'Homeland Security', 'State', 'Housing and Urban Development', 'Agriculture', 'Labor', 'Transportation', 'Defense', 'Treasury', 'Health and Human Services']}, {'Creation': ['1979', '1966', '1889', '1977', '1965', '1947', '1903', '1953', '1913', '1870', '1989', '1789', '1849', '2002']}, {'Department_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}, {'Ranking': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}, {'Budget_in_Billions': [439.3, 6.2, 9.96, 10.7, 11.1, 73.2, 77.6, 46.2, 44.6, 21.

In [5]:
from seq2seq_construction.text2cypher_dataset_builder import Text2Cypher
data_builder = Text2Cypher()

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
for split in ['train.json', 'dev.json']:
    dev_fpath = os.getcwd() + '/data/text2cypher/'+ split
    data_builder._generate_examples(dev_fpath, os.getcwd() + '/data/text2cypher/schema.json')

In [7]:
data_builder._info()

DatasetInfo(description='Spider is a large-scale complex and cross-domain semantic parsing and text-toSQL dataset annotated by 11 college students.\nWe use a dataset transformed from a subset of Spider to train our Text-to-Cypher semantic parsing.\n', citation='@article{yu2018spider,\n  title={Spider: A large-scale human-labeled dataset for complex and cross-domain semantic parsing and text-to-sql task},\n  author={Yu, Tao and Zhang, Rui and Yang, Kai and Yasunaga, Michihiro and Wang, Dongxu and Li, Zifan and Ma, James and Li, Irene and Yao, Qingning and Roman, Shanelle and others},\n  journal={arXiv preprint arXiv:1809.08887},\n  year={2018}\n}\n\n@article{raffel2020exploring,\n  title={Exploring the limits of transfer learning with a unified text-to-text transformer},\n  author={Raffel, Colin and Shazeer, Noam and Roberts, Adam and Lee, Katherine and Narang, Sharan and Matena, Michael and Zhou, Yanqi and Li, Wei and Liu, Peter J},\n  journal={The Journal of Machine Learning Research}

### Test DB content encoding --> done!

In [8]:
import sqlite3
db_path = '/home/22842219/Desktop/projects_data/spider/database/department_management/department_management.sqlite'
table_name = 'head'
column_name = 'born_state'
question = 'What are the names of the heads who are born outside the California state?'

In [9]:
# picklist saves all the 'name' column values
# test column values picklist
def get_column_picklist(table_name: str, column_name: str, db_path: str) -> list:
    fetch_sql = "SELECT DISTINCT `{}` FROM `{}`".format(column_name, table_name)
    try:
        conn = sqlite3.connect(db_path)
        conn.text_factory = bytes
        c = conn.cursor()
        c.execute(fetch_sql)
        picklist = set()
        for x in c.fetchall():
            if isinstance(x[0], str):
                picklist.add(x[0].encode("utf-8"))
            elif isinstance(x[0], bytes):
                try:
                    picklist.add(x[0].decode("utf-8"))
                except UnicodeDecodeError:
                    picklist.add(x[0].decode("latin-1"))
            else:
                picklist.add(x[0])
        picklist = list(picklist)
    finally:
        conn.close()
    return picklist

picklist = get_column_picklist(table_name, column_name, db_path)
print(picklist)

['Florida', 'Connecticut', 'Alabama', 'California', 'Delaware']


In [10]:
from seq2seq_construction.bridge_content_encoder import is_stopword, is_number, is_common_db_term, get_effective_match_source, prefix_match, is_commonword, split

In [11]:
import difflib
from typing import List, Optional, Tuple
from rapidfuzz import fuzz
import sqlite3
import functools

In [12]:
def get_matched_entries(
    s: str, field_values: List[str], m_theta: float = 0.85, s_theta: float = 0.85
) -> Optional[List[Tuple[str, Tuple[str, str, float, float, int]]]]:
    if not field_values:
        return None

    if isinstance(s, str):
        n_grams = split(s)
    else:
        n_grams = s

    matched = dict()
    print(f'n_grams: {n_grams}')
    for id, field_value in enumerate(field_values):
        if not isinstance(field_value, str):
            continue
        fv_tokens = split(field_value)   
        sm = difflib.SequenceMatcher(None, n_grams, fv_tokens)
        match = sm.find_longest_match(0, len(n_grams), 0, len(fv_tokens))
        if match.size > 0:
            source_match = get_effective_match_source(
                n_grams, match.a, match.a + match.size
            )

            if source_match and source_match.size > 1:

                print(f'{id}, field_value: {field_value}, fv_tokens: {fv_tokens}, n_grams: {n_grams}')
                print(f'sm: {sm}, match: {match}')
                print(f'match_start: {match.a}, match_end:{match.size + match.a}')
                print(n_grams[match.a])
                print(f'source_match: {source_match}')
                print(field_value[match.b : match.b + match.size])

                match_str = field_value[match.b : match.b + match.size]
                source_match_str = s[
                    source_match.start : source_match.start + source_match.size
                ]
                c_match_str = match_str.lower().strip()
                c_source_match_str = source_match_str.lower().strip()
                c_field_value = field_value.lower().strip()
                if (
                    c_match_str
                    and not is_number(c_match_str)
                    and not is_common_db_term(c_match_str)
                ):
                    if (
                        is_stopword(c_match_str)
                        or is_stopword(c_source_match_str)
                        or is_stopword(c_field_value)
                    ):
                        continue
                    if c_source_match_str.endswith(c_match_str + "'s"):
                        match_score = 1.0
                    else:
                        if prefix_match(c_field_value, c_source_match_str):
                            match_score = (
                                fuzz.ratio(c_field_value, c_source_match_str) / 100
                            )
                        else:
                            match_score = 0
                    if (
                        is_commonword(c_match_str)
                        or is_commonword(c_source_match_str)
                        or is_commonword(c_field_value)
                    ) and match_score < 1:
                        continue
                    s_match_score = match_score
                    if match_score >= m_theta and s_match_score >= s_theta:
                        if field_value.isupper() and match_score * s_match_score < 1:
                            continue
                        matched[match_str] = (
                            field_value,
                            source_match_str,
                            match_score,
                            s_match_score,
                            match.size,
                        )
    # for understanding process
    for match_str, value in matched.items():
        print(f'match_str: {match_str}')
        print(f'(field_value, source_match_str, match_score, s_match_score, match.size):  {value}')
    # for understanding process 

    if not matched:
        return None
    else:
        return sorted(
            matched.items(),
            key=lambda x: (1e16 * x[1][2] + 1e8 * x[1][3] + x[1][4]),
            reverse=True,
        )



In [13]:
matches_entries = get_matched_entries(question, picklist, m_theta = 0.85, s_theta = 0.85)
print(matches_entries)

n_grams: ['w', 'h', 'a', 't', ' ', 'a', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'n', 'a', 'm', 'e', 's', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'h', 'e', 'a', 'd', 's', ' ', 'w', 'h', 'o', ' ', 'a', 'r', 'e', ' ', 'b', 'o', 'r', 'n', ' ', 'o', 'u', 't', 's', 'i', 'd', 'e', ' ', 't', 'h', 'e', ' ', 'c', 'a', 'l', 'i', 'f', 'o', 'r', 'n', 'i', 'a', ' ', 's', 't', 'a', 't', 'e', '?']
3, field_value: California, fv_tokens: ['c', 'a', 'l', 'i', 'f', 'o', 'r', 'n', 'i', 'a'], n_grams: ['w', 'h', 'a', 't', ' ', 'a', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'n', 'a', 'm', 'e', 's', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'h', 'e', 'a', 'd', 's', ' ', 'w', 'h', 'o', ' ', 'a', 'r', 'e', ' ', 'b', 'o', 'r', 'n', ' ', 'o', 'u', 't', 's', 'i', 'd', 'e', ' ', 't', 'h', 'e', ' ', 'c', 'a', 'l', 'i', 'f', 'o', 'r', 'n', 'i', 'a', ' ', 's', 't', 'a', 't', 'e', '?']
sm: <difflib.SequenceMatcher object at 0x7fd9d219ea30>, match: Match(a=57, b=0, size=10)
match_start: 57, match_end:67
c
source_match: <seq2seq_const

In [14]:
def get_database_matches(
    question: str,
    table_name: str,
    column_name: str,
    db_path: str,
    top_k_matches: int = 2,
    match_threshold: float = 0.85,
) -> List[str]:
    picklist = get_column_picklist(
        table_name=table_name, column_name=column_name, db_path=db_path
    )
    print(f'picklist: {picklist}, question: {type(question)}, {question}')
    matches = []
    if picklist and isinstance(picklist[0], str):
        matched_entries = get_matched_entries(
            s=question,
            field_values=picklist,
            m_theta=match_threshold,
            s_theta=match_threshold,
        )
        if matched_entries:
            num_values_inserted = 0
            for _match_str, (
                field_value,
                _s_match_str,
                match_score,
                s_match_score,
                _match_size,
            ) in matched_entries:
                if "name" in column_name and match_score * s_match_score < 1:
                    continue
                if table_name != "sqlite_sequence":  # Spider database artifact
                    matches.append(field_value)
                    num_values_inserted += 1
                    if num_values_inserted >= top_k_matches:
                        break
    return matches

In [15]:
matches = get_database_matches(question, table_name, column_name, db_path)
print(matches)

picklist: ['Florida', 'Connecticut', 'Alabama', 'California', 'Delaware'], question: <class 'str'>, What are the names of the heads who are born outside the California state?
n_grams: ['w', 'h', 'a', 't', ' ', 'a', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'n', 'a', 'm', 'e', 's', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'h', 'e', 'a', 'd', 's', ' ', 'w', 'h', 'o', ' ', 'a', 'r', 'e', ' ', 'b', 'o', 'r', 'n', ' ', 'o', 'u', 't', 's', 'i', 'd', 'e', ' ', 't', 'h', 'e', ' ', 'c', 'a', 'l', 'i', 'f', 'o', 'r', 'n', 'i', 'a', ' ', 's', 't', 'a', 't', 'e', '?']
3, field_value: California, fv_tokens: ['c', 'a', 'l', 'i', 'f', 'o', 'r', 'n', 'i', 'a'], n_grams: ['w', 'h', 'a', 't', ' ', 'a', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'n', 'a', 'm', 'e', 's', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'h', 'e', 'a', 'd', 's', ' ', 'w', 'h', 'o', ' ', 'a', 'r', 'e', ' ', 'b', 'o', 'r', 'n', ' ', 'o', 'u', 't', 's', 'i', 'd', 'e', ' ', 't', 'h', 'e', ' ', 'c', 'a', 'l', 'i', 'f', 'o', 'r', 'n', 'i', 'a', ' ', 's',

In [5]:
from seq2seq_construction.bridge_content_encoder import get_database_matches, get_property_picklist
# test ours DB content encoding
import codecs, json
schema_path = '/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher/schema.json'
db_id = 'department_management'
tag_name = ":`department_management.head`"
property_name = 'born_state'
question = 'What are the names of the heads who are born outside the California state?'
print(f'question: {question}')
print(f'tag_name: {tag_name}')
print(f'property_name: {property_name}')


question: What are the names of the heads who are born outside the California state?
tag_name: :`department_management.head`
property_name: born_state


In [6]:
def get_property_picklist(db_id: str, tag_name: str, property_name: str, schema_path: str) -> list:
    with codecs.open(schema_path, 'r') as f:
        schema = json.load(f)
        every_schema=schema[db_id]
        for property_value_pair in every_schema[tag_name]:
            print(property_value_pair)
            
            if property_name in list(property_value_pair.keys()):
                picklist = property_value_pair[property_name]
    return picklist

In [7]:
text2cypher_picklist = get_property_picklist(db_id, tag_name, property_name, schema_path)
print(f'text2cypher_picklist: {text2cypher_picklist}')

{'name': ['Tiger Woods', 'Billy Mayfair', 'Jeff Maggert', 'K. J. Choi', 'Franklin Langham', 'Nick Faldo', 'Pádraig Harrington', 'Dudley Hart', 'Sergio García', 'Stewart Cink']}
{'born_state': ['Delaware', 'Alabama', 'Connecticut', 'California', 'Florida']}
{'age': [67.0, 68.0, 69.0, 43.0, 50.0, 52.0, 53.0, 56.0]}
{'head_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
text2cypher_picklist: ['Delaware', 'Alabama', 'Connecticut', 'California', 'Florida']


In [8]:
matches = get_database_matches(db_id, question, tag_name, property_name, schema_path)
print(matches)

{'name': ['Tiger Woods', 'Billy Mayfair', 'Jeff Maggert', 'K. J. Choi', 'Franklin Langham', 'Nick Faldo', 'Pádraig Harrington', 'Dudley Hart', 'Sergio García', 'Stewart Cink']}
{'born_state': ['Delaware', 'Alabama', 'Connecticut', 'California', 'Florida']}
{'age': [67.0, 68.0, 69.0, 43.0, 50.0, 52.0, 53.0, 56.0]}
{'head_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
picklist: ['Delaware', 'Alabama', 'Connecticut', 'California', 'Florida'], question: <class 'str'>, What are the names of the heads who are born outside the California state?
n_grams: ['w', 'h', 'a', 't', ' ', 'a', 'r', 'e', ' ', 't', 'h', 'e', ' ', 'n', 'a', 'm', 'e', 's', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'h', 'e', 'a', 'd', 's', ' ', 'w', 'h', 'o', ' ', 'a', 'r', 'e', ' ', 'b', 'o', 'r', 'n', ' ', 'o', 'u', 't', 's', 'i', 'd', 'e', ' ', 't', 'h', 'e', ' ', 'c', 'a', 'l', 'i', 'f', 'o', 'r', 'n', 'i', 'a', ' ', 's', 't', 'a', 't', 'e', '?']
fv_tokens:
fv_tokens:
fv_tokens:
fv_tokens:
fv_tokens:
['California']


### Test input serialization

In [ ]:

import os, torch
from tqdm import tqdm
from copy import deepcopy
import datasets




In [ ]:
### test text2sql
loader_path = '/home/22842219/Desktop/openSource/UnifiedSKGG-subSpider/tasks/spider.py'
data_store_path ='/home/22842219/Desktop/openSource/UnifiedSKGG-subSpider/data'
cache_root = os.path.join('/home/22842219/Desktop/openSource/UnifiedSKGG-subSpider', '/out/cache')


In [ ]:
loader_path = 'seq2seq_construction/text2cypher_dataset_builder.py'
data_store_path = 'data'
raw_datasets_split: datasets.DatasetDict = datasets.load_dataset(path=loader_path, cache_dir=data_store_path)



In [ ]:
cache_root = os.getcwd() + '/output/cache'
cache_path = os.path.join(cache_root, 'spider_dev.cache')

if os.path.exists(cache_path):
    extended_data = torch.load(cache_path)
else:
    extended_data = []
    for raw_data in tqdm(raw_datasets):
        extend_data = deepcopy(raw_data)